In [20]:
from IPython.display import display, clear_output # Usado para renderizar objetos jupyter notebook.
import ipywidgets as widgets # Biblioteca para criar widgets jupyter notebook.
from datetime import datetime
from enum import Enum
from abc import ABC, abstractmethod # Usado na implementação de classes abstratas
import pandas as pd # Biblioteca para manipulação e análise de dados
import matplotlib.pyplot as plt # Biblioteca para criação de gráficos
import sqlite3 # Comunicação com bancos de dados SQLite
from sqlite3 import Error

In [21]:
""" Classes do Sistema """

class CategoriaMovimentacao(Enum):
    RECEITA = "Receita"
    DESPESA = "Despesa"

class TipoMovimentacao:
    def __init__(self, id: int, nome: str, categoria: CategoriaMovimentacao, descricao: str = ""):
        self.id = id
        self.nome = nome
        self.categoria = categoria
        self.descricao = descricao

    def __str__(self):
        return f"{self.nome} ({self.categoria.value})"

class Movimentacao(ABC):
    def __init__(self, id: int, valor: float, data: datetime, descricao: str, conta: 'Conta', tipo_movimentacao: TipoMovimentacao):
        self.id = id
        self.valor = valor
        self.data = data
        self.descricao = descricao
        self.conta = conta
        self.tipo_movimentacao = tipo_movimentacao

    @abstractmethod
    def executar(self):
        pass

class MovimentacaoSimples(Movimentacao):
    def executar(self):
        if self.tipo_movimentacao.categoria == CategoriaMovimentacao.DESPESA:
            return self.conta.debitar(self.valor)
        elif self.tipo_movimentacao.categoria == CategoriaMovimentacao.RECEITA:
            return self.conta.creditar(self.valor)
        return f"Movimentação {self.tipo_movimentacao.nome} realizada: R${self.valor:.2f}"

class Transferencia(Movimentacao):
    def __init__(self, id: int, valor: float, data: datetime, descricao: str, conta_origem: 'Conta',
                 conta_destino: 'Conta', tipo_movimentacao: TipoMovimentacao):
        super().__init__(id, valor, data, descricao, conta_origem, tipo_movimentacao)
        self.conta_destino = conta_destino

    def executar(self):
        try:
            resultado_origem = self.conta.debitar(self.valor)
            resultado_destino = self.conta_destino.creditar(self.valor)
            return f"Transferência realizada:\n{resultado_origem}\n{resultado_destino}"
        except Exception as e:
            return f"Erro na transferência: {str(e)}"

class Conta(ABC):
    def __init__(self, id: int, nome: str, descricao: str, saldo: float = 0.0):
        self.id = id
        self.nome = nome
        self.descricao = descricao
        self.saldo = saldo
        self.movimentacoes = []

    def creditar(self, valor: float):
        self.saldo += valor
        return f"Crédito de R${valor:.2f} realizado. Saldo atual: R${self.saldo:.2f}"

    def debitar(self, valor: float):
        if self.saldo >= valor:
            self.saldo -= valor
            return f"Débito de R${valor:.2f} realizado. Saldo atual: R${self.saldo:.2f}"
        raise ValueError("Saldo insuficiente")

    def adicionar_movimentacao(self, movimentacao: Movimentacao):
        self.movimentacoes.append(movimentacao)
        return movimentacao.executar()

    @abstractmethod
    def get_saldo_disponivel(self) -> float:
        pass

    def resumo_movimentacoes(self):
        dados = []
        for mov in self.movimentacoes:
            dados.append({
                'Data': mov.data.strftime('%d/%m/%Y'),
                'Tipo': mov.tipo_movimentacao.nome,
                'Valor': mov.valor,
                'Descrição': mov.descricao,
                'Categoria': mov.tipo_movimentacao.categoria.value
            })
        return pd.DataFrame(dados)

class ContaCorrente(Conta):
    def get_saldo_disponivel(self) -> float:
        return self.saldo

In [22]:
""" Banco de Dados SQLite """
class Database:
    def __init__(self, db_file="financas.db"):
        self.db_file = db_file
        self.conn = None
        self.criar_tabelas()

    def conectar(self):
        try:
            self.conn = sqlite3.connect(self.db_file)
            return self.conn
        except Error as e:
            print(e)
        return None

    def desconectar(self):
        if self.conn:
            self.conn.close()

    def criar_tabelas(self):
        sql_contas = """CREATE TABLE IF NOT EXISTS contas (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nome TEXT NOT NULL,
            descricao TEXT,
            saldo REAL DEFAULT 0,
            tipo TEXT DEFAULT 'corrente',
            parametro_extra REAL DEFAULT 0
        );"""

        sql_tipos_movimentacao = """CREATE TABLE IF NOT EXISTS tipos_movimentacao (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nome TEXT NOT NULL,
            categoria TEXT NOT NULL,
            descricao TEXT
        );"""

        sql_movimentacoes = """CREATE TABLE IF NOT EXISTS movimentacoes (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            conta_id INTEGER NOT NULL,
            tipo_id INTEGER NOT NULL,
            valor REAL NOT NULL,
            data TEXT NOT NULL,
            descricao TEXT,
            conta_destino_id INTEGER,
            parcelas INTEGER DEFAULT 1,
            intervalo_dias INTEGER DEFAULT 0,
            FOREIGN KEY (conta_id) REFERENCES contas (id),
            FOREIGN KEY (tipo_id) REFERENCES tipos_movimentacao (id)
        );"""

        conn = self.conectar()
        if conn is not None:
            try:
                c = conn.cursor()
                c.execute(sql_contas)
                c.execute(sql_tipos_movimentacao)
                c.execute(sql_movimentacoes)
                conn.commit()
            except Error as e:
                print(e)
            finally:
                self.desconectar()

    def inserir_conta(self, conta):
        sql = """INSERT INTO contas(nome, descricao, saldo, tipo, parametro_extra) VALUES(?,?,?,?,?)"""
        conn = self.conectar()
        if conn is not None:
            try:
                tipo = 'corrente' if isinstance(conta, ContaCorrente) else 'investimento'
                parametro = conta.limite if hasattr(conta, 'limite') else conta.rendimento_anual if hasattr(conta, 'rendimento_anual') else 0

                c = conn.cursor()
                c.execute(sql, (conta.nome, conta.descricao, conta.saldo, tipo, parametro))
                conn.commit()
                return c.lastrowid
            except Error as e:
                print(e)
            finally:
                self.desconectar()
        return None

    def inserir_tipo_movimentacao(self, tipo):
        sql = """INSERT INTO tipos_movimentacao(nome, categoria, descricao) VALUES(?,?,?)"""
        conn = self.conectar()
        if conn is not None:
            try:
                c = conn.cursor()
                c.execute(sql, (tipo.nome, tipo.categoria.value, tipo.descricao))
                conn.commit()
                return c.lastrowid
            except Error as e:
                print(e)
            finally:
                self.desconectar()
        return None

    def inserir_movimentacao(self, movimentacao):
        sql = """INSERT INTO movimentacoes(conta_id, tipo_id, valor, data, descricao, conta_destino_id, parcelas, intervalo_dias)
                 VALUES(?,?,?,?,?,?,?,?)"""
        conn = self.conectar()
        if conn is not None:
            try:
                conta_destino_id = movimentacao.conta_destino.id if isinstance(movimentacao, Transferencia) else None
                parcelas = movimentacao.parcelas if hasattr(movimentacao, 'parcelas') else 1
                intervalo_dias = movimentacao.intervalo_dias if hasattr(movimentacao, 'intervalo_dias') else 0

                c = conn.cursor()
                c.execute(sql, (
                    movimentacao.conta.id,
                    movimentacao.tipo_movimentacao.id,
                    movimentacao.valor,
                    movimentacao.data.isoformat(),
                    movimentacao.descricao,
                    conta_destino_id,
                    parcelas,
                    intervalo_dias
                ))
                conn.commit()
                return c.lastrowid
            except Error as e:
                print(e)
            finally:
                self.desconectar()
        return None

    def carregar_contas(self):
        sql = "SELECT * FROM contas"
        conn = self.conectar()
        if conn is not None:
            try:
                c = conn.cursor()
                c.execute(sql)
                rows = c.fetchall()

                contas = []
                for row in rows:
                    if row[4] == 'corrente':
                        conta = ContaCorrente(row[0], row[1], row[2], row[3])
                        conta.limite = row[5]
                    else:
                        conta = ContaCorrente(row[0], row[1], row[2], row[3])
                        conta.rendimento_anual = row[5]
                    contas.append(conta)

                return contas
            except Error as e:
                print(e)
            finally:
                self.desconectar()
        return []

    def carregar_tipos_movimentacao(self):
        sql = "SELECT * FROM tipos_movimentacao"
        conn = self.conectar()
        if conn is not None:
            try:
                c = conn.cursor()
                c.execute(sql)
                rows = c.fetchall()

                tipos = []
                for row in rows:
                    categoria = CategoriaMovimentacao(row[2])
                    tipos.append(TipoMovimentacao(row[0], row[1], categoria, row[3]))

                return tipos
            except Error as e:
                print(e)
            finally:
                self.desconectar()
        return []

    def carregar_movimentacoes(self, sistema):
        sql = "SELECT * FROM movimentacoes"
        conn = self.conectar()
        if conn is not None:
            try:
                c = conn.cursor()
                c.execute(sql)
                rows = c.fetchall()

                movimentacoes = []
                for row in rows:
                    conta = sistema.get_conta_por_id(row[1])
                    tipo = sistema.get_tipo_movimentacao_por_id(row[2])
                    data = datetime.fromisoformat(row[4])

                    if row[6]:  # Se tem conta_destino_id, é transferência
                        conta_destino = sistema.get_conta_por_id(row[6])
                        mov = Transferencia(
                            row[0], row[3], data, row[5], conta, conta_destino, tipo
                        )
                    elif row[7] > 1:  # Se tem parcelas > 1, é parcelada
                        mov = MovimentacaoSimples(
                            row[0], row[3], data, row[5], conta, tipo
                        )
                    else:
                        mov = MovimentacaoSimples(
                            row[0], row[3], data, row[5], conta, tipo
                        )

                    conta.movimentacoes.append(mov)
                    movimentacoes.append(mov)

                return movimentacoes
            except Error as e:
                print(e)
            finally:
                self.desconectar()
        return []

    def atualizar_conta(self, conta):
        sql = """UPDATE contas SET nome = ?, descricao = ?, saldo = ?, parametro_extra = ? WHERE id = ?"""
        conn = self.conectar()
        if conn is not None:
            try:
                parametro = conta.limite if hasattr(conta, 'limite') else conta.rendimento_anual if hasattr(conta, 'rendimento_anual') else 0

                c = conn.cursor()
                c.execute(sql, (conta.nome, conta.descricao, conta.saldo, parametro, conta.id))
                conn.commit()
                return True
            except Error as e:
                print(e)
                return False
            finally:
                self.desconectar()
        return False

    def atualizar_tipo_movimentacao(self, tipo):
        sql = """UPDATE tipos_movimentacao SET nome = ?, categoria = ?, descricao = ? WHERE id = ?"""
        conn = self.conectar()
        if conn is not None:
            try:
                c = conn.cursor()
                c.execute(sql, (tipo.nome, tipo.categoria.value, tipo.descricao, tipo.id))
                conn.commit()
                return True
            except Error as e:
                print(e)
                return False
            finally:
                self.desconectar()
        return False

    def atualizar_movimentacao(self, movimentacao):
        sql = """UPDATE movimentacoes SET valor = ?, descricao = ? WHERE id = ?"""
        conn = self.conectar()
        if conn is not None:
            try:
                c = conn.cursor()
                c.execute(sql, (movimentacao.valor, movimentacao.descricao, movimentacao.id))
                conn.commit()
                return True
            except Error as e:
                print(e)
                return False
            finally:
                self.desconectar()
        return False

    def remover_conta(self, id_conta):
        sql = "DELETE FROM contas WHERE id = ?"
        conn = self.conectar()
        if conn is not None:
            try:
                c = conn.cursor()
                c.execute(sql, (id_conta,))
                conn.commit()
                return c.rowcount > 0
            except Error as e:
                print(e)
                return False
            finally:
                self.desconectar()
        return False

    def remover_tipo_movimentacao(self, id_tipo):
        sql = "DELETE FROM tipos_movimentacao WHERE id = ?"
        conn = self.conectar()
        if conn is not None:
            try:
                c = conn.cursor()
                c.execute(sql, (id_tipo,))
                conn.commit()
                return c.rowcount > 0
            except Error as e:
                print(e)
                return False
            finally:
                self.desconectar()
        return False

    def remover_movimentacao(self, id_movimentacao):
        sql = "DELETE FROM movimentacoes WHERE id = ?"
        conn = self.conectar()
        if conn is not None:
            try:
                c = conn.cursor()
                c.execute(sql, (id_movimentacao,))
                conn.commit()
                return c.rowcount > 0
            except Error as e:
                print(e)
                return False
            finally:
                self.desconectar()
        return False

In [23]:
class SistemaFinanceiro:
    def __init__(self):
        self.db = Database()
        self.contas = self.db.carregar_contas()
        self.tipos_movimentacao = self.db.carregar_tipos_movimentacao()
        self._inicializar_tipos_padrao()
        self.db.carregar_movimentacoes(self)

    def _inicializar_tipos_padrao(self):
        if not any(t.nome == "Salário" for t in self.tipos_movimentacao):
            self.criar_tipo_movimentacao("Salário", CategoriaMovimentacao.RECEITA)
        if not any(t.nome == "Alimentação" for t in self.tipos_movimentacao):
            self.criar_tipo_movimentacao("Alimentação", CategoriaMovimentacao.DESPESA)
        if not any(t.nome == "Educação" for t in self.tipos_movimentacao):
            self.criar_tipo_movimentacao("Educação", CategoriaMovimentacao.DESPESA)
        if not any(t.nome == "Saúde e Bem-Estar" for t in self.tipos_movimentacao):
            self.criar_tipo_movimentacao("Saúde e Bem-Estar", CategoriaMovimentacao.DESPESA)
        if not any(t.nome == "Transporte" for t in self.tipos_movimentacao):
            self.criar_tipo_movimentacao("Transporte", CategoriaMovimentacao.DESPESA)
        if not any(t.nome == "Serviços" for t in self.tipos_movimentacao):
            self.criar_tipo_movimentacao("Serviços", CategoriaMovimentacao.DESPESA)
        if not any(t.nome == "Lazer" for t in self.tipos_movimentacao):
            self.criar_tipo_movimentacao("Lazer", CategoriaMovimentacao.DESPESA)
        if not any(t.nome == "Transferência" for t in self.tipos_movimentacao):
            self.criar_tipo_movimentacao("Transferência", CategoriaMovimentacao.DESPESA)

    def criar_conta(self, nome: str, descricao: str, saldo: float = 0.0) -> Conta:
        nova_id = len(self.contas) + 1
        conta = ContaCorrente(nova_id, nome, descricao, saldo)
        conta.id = self.db.inserir_conta(conta)
        if conta.id:
            self.contas.append(conta)
        return conta

    def criar_tipo_movimentacao(self, nome: str, categoria: CategoriaMovimentacao, descricao: str = "") -> TipoMovimentacao:
        novo_id = len(self.tipos_movimentacao) + 1
        novo_tipo = TipoMovimentacao(novo_id, nome, categoria, descricao)
        novo_tipo.id = self.db.inserir_tipo_movimentacao(novo_tipo)
        if novo_tipo.id:
            self.tipos_movimentacao.append(novo_tipo)
        return novo_tipo

    def get_conta_por_id(self, id: int) -> Conta:
        for conta in self.contas:
            if conta.id == id:
                return conta
        return None

    def get_conta_por_nome(self, nome: str) -> Conta:
        for conta in self.contas:
            if conta.nome == nome:
                return conta
        return None

    def get_tipo_movimentacao_por_id(self, id: int) -> TipoMovimentacao:
        for tipo in self.tipos_movimentacao:
            if tipo.id == id:
                return tipo
        return None

    def get_tipo_movimentacao_por_nome(self, nome: str) -> TipoMovimentacao:
        for tipo in self.tipos_movimentacao:
            if tipo.nome == nome:
                return tipo
        return None

    def transferir_entre_contas(self, conta_origem: str, conta_destino: str, valor: float, descricao: str = ""):
        origem = self.get_conta_por_nome(conta_origem)
        destino = self.get_conta_por_nome(conta_destino)
        
        if not origem:
            return "Conta de origem não encontrada"
        if not destino:
            return "Conta de destino não encontrada"
        if origem == destino:
            return "Conta de origem e destino devem ser diferentes"
        if valor <= 0:
            return "O valor da transferência deve ser positivo"

        # Obter ou criar o tipo de movimentação para transferência
        tipo_transferencia = self.get_tipo_movimentacao_por_nome("Transferência")
        if not tipo_transferencia:
            tipo_transferencia = self.criar_tipo_movimentacao("Transferência", CategoriaMovimentacao.DESPESA)

        try:
            transferencia = Transferencia(
                len(origem.movimentacoes) + 1,
                valor,
                datetime.now(),
                descricao,
                origem,
                destino,
                tipo_transferencia
            )

            transferencia.id = self.db.inserir_movimentacao(transferencia)
            if transferencia.id:
                origem.movimentacoes.append(transferencia)
                resultado = transferencia.executar()
                self.db.atualizar_conta(origem)
                self.db.atualizar_conta(destino)
                return resultado
            return "Erro ao registrar transferência no banco de dados"
        except ValueError as e:
            return str(e)
        except Exception as e:
            return f"Erro inesperado na transferência: {str(e)}"

    def editar_conta(self, b):
      with self.output:
          clear_output()
          try:
              conta_id = self.dropdown_contas.value
              conta = self.sistema.get_conta_por_id(conta_id)

              if conta:
                  novo_nome = self.novo_nome_conta.value or conta.nome
                  nova_descricao = self.nova_descricao_conta.value or conta.descricao
                  novo_saldo = self.novo_saldo_conta.value if self.novo_saldo_conta.value != 0 else conta.saldo

                  if self.sistema.editar_conta(conta_id, novo_nome, nova_descricao, novo_saldo):
                      print("Conta editada com sucesso!")
                      self.atualizar_dropdowns()
                  else:
                      print("Erro ao editar conta.")
              else:
                  print("Conta não encontrada!")
          except Exception as e:
              print(f"Erro ao editar conta: {str(e)}")

    def editar_tipo_movimentacao(self, b):
        with self.output:
            clear_output()
            try:
                tipo_id = self.dropdown_tipos.value
                tipo = self.sistema.get_tipo_movimentacao_por_id(tipo_id)

                if tipo:
                    novo_nome = self.novo_nome_tipo.value or tipo.nome
                    nova_categoria = CategoriaMovimentacao(self.nova_categoria_tipo.value) if self.nova_categoria_tipo.value else tipo.categoria
                    nova_descricao = self.nova_descricao_tipo.value or tipo.descricao

                    if self.sistema.editar_tipo_movimentacao(tipo_id, novo_nome, nova_categoria, nova_descricao):
                        print("Tipo de movimentação editado com sucesso!")
                        self.atualizar_dropdowns()
                    else:
                        print("Erro ao editar tipo de movimentação.")
                else:
                    print("Tipo não encontrado!")
            except Exception as e:
                print(f"Erro ao editar tipo: {str(e)}")

    def editar_movimentacao(self, b):
        with self.output:
            clear_output()
            try:
                mov_id = self.dropdown_movimentacoes.value
                novo_valor = self.novo_valor_mov.value
                nova_descricao = self.nova_descricao_mov.value

                if novo_valor <= 0:
                    print("O valor deve ser maior que zero")
                    return

                if self.sistema.editar_movimentacao(mov_id, novo_valor, nova_descricao):
                    print("Movimentação editada com sucesso!")
                    self.atualizar_dropdowns()
                else:
                    print("Erro ao editar movimentação.")
            except Exception as e:
                print(f"Erro ao editar movimentação: {str(e)}")

    def remover_conta(self, id_conta: int):
        conta = self.get_conta_por_id(id_conta)
        if conta:
            if conta.movimentacoes:
                return False # Não permite remover contas com movimentações
            self.contas.remove(conta)
            return self.db.remover_conta(id_conta)
        return False

    def remover_tipo_movimentacao(self, id_tipo: int):
        tipo = self.get_tipo_movimentacao_por_id(id_tipo)
        if tipo:
            for conta in self.contas:
                for mov in conta.movimentacoes:
                    if mov.tipo_movimentacao.id == id_tipo:
                        return False  # Não permite remover tipos em uso
            self.tipos_movimentacao.remove(tipo)
            return self.db.remover_tipo_movimentacao(id_tipo)
        return False

    def remover_movimentacao(self, id_movimentacao: int):
        for conta in self.contas:
            for mov in conta.movimentacoes:
                if mov.id == id_movimentacao:
                    # Reverte o efeito da movimentação
                    if mov.tipo_movimentacao.categoria == CategoriaMovimentacao.RECEITA:
                        conta.saldo -= mov.valor
                    elif mov.tipo_movimentacao.categoria == CategoriaMovimentacao.DESPESA:
                        conta.saldo += mov.valor

                    conta.movimentacoes.remove(mov)
                    return self.db.remover_movimentacao(id_movimentacao)
        return False

In [24]:
class Financas:
    def __init__(self):
        self.sistema = SistemaFinanceiro()
        self.criar_widgets()
        self.exibir_painel_principal()

    def criar_widgets(self):
        # Widgets para Contas
        self.nome_conta = widgets.Text(description='Nome:')
        self.descricao_conta = widgets.Text(description='Descrição:')
        self.saldo_inicial = widgets.FloatText(value=0.0, description='Saldo:')
        self.btn_criar_conta = widgets.Button(description='Criar Conta', button_style='success')

        # Widgets para seleção de conta para edição/remoção
        self.dropdown_contas = widgets.Dropdown(description='Selecionar Conta:')
        self.btn_editar_conta = widgets.Button(description='Editar', button_style='info')
        self.btn_remover_conta = widgets.Button(description='Remover', button_style='danger')

        # Widgets para edição de conta
        self.novo_nome_conta = widgets.Text(description='Novo nome:')
        self.nova_descricao_conta = widgets.Text(description='Nova descrição:')
        self.novo_saldo_conta = widgets.FloatText(description='Novo saldo:')

        # Widgets para Tipos de Movimentação
        self.nome_tipo = widgets.Text(description='Nome:')
        self.categoria_tipo = widgets.Dropdown(
            options=[cat.value for cat in CategoriaMovimentacao],
            description='Categoria:'
        )
        self.descricao_tipo = widgets.Text(description='Descrição:')
        self.btn_criar_tipo = widgets.Button(description='Criar Tipo', button_style='success')

        # Widgets para seleção de tipo para edição/remoção
        self.dropdown_tipos = widgets.Dropdown(description='Selecionar Tipo:')
        self.btn_editar_tipo = widgets.Button(description='Editar', button_style='info')
        self.btn_remover_tipo = widgets.Button(description='Remover', button_style='danger')

        # Widgets para edição de tipo
        self.novo_nome_tipo = widgets.Text(description='Novo nome:')
        self.nova_categoria_tipo = widgets.Dropdown(
            options=[cat.value for cat in CategoriaMovimentacao],
            description='Nova categoria:'
        )
        self.nova_descricao_tipo = widgets.Text(description='Nova descrição:')

        # Widgets para Movimentações
        self.conta_mov = widgets.Dropdown(description='Conta:')
        self.tipo_mov = widgets.Dropdown(description='Tipo:')
        self.valor_mov = widgets.FloatText(value=0.0, description='Valor:')
        self.descricao_mov = widgets.Text(description='Descrição:')
        self.btn_registrar_mov = widgets.Button(description='Registrar', button_style='success')

        # Widgets para seleção de movimentação para edição/remoção
        self.dropdown_movimentacoes = widgets.Dropdown(description='Selecionar Movimentação:')
        self.btn_editar_mov = widgets.Button(description='Editar', button_style='info')
        self.btn_remover_mov = widgets.Button(description='Remover', button_style='danger')

        # Widgets para edição de movimentação
        self.novo_valor_mov = widgets.FloatText(description='Novo valor:')
        self.nova_descricao_mov = widgets.Text(description='Nova descrição:')

        # Widgets para Transferências
        self.conta_origem = widgets.Dropdown(description='Origem:')
        self.conta_destino = widgets.Dropdown(description='Destino:')
        self.valor_transf = widgets.FloatText(value=0.0, description='Valor:')
        self.descricao_transf = widgets.Text(description='Descrição:')
        self.btn_transferir = widgets.Button(description='Transferir', button_style='success')

        # Widgets para Relatórios
        self.btn_rel_contas = widgets.Button(description='Mostrar Contas')
        self.btn_rel_tipos = widgets.Button(description='Mostrar Tipos')
        self.btn_rel_mov = widgets.Button(description='Mostrar Movimentações')

        # Configura funções
        self.btn_criar_conta.on_click(self.criar_conta)
        self.btn_editar_conta.on_click(self.editar_conta)
        self.btn_remover_conta.on_click(self.remover_conta)
        self.btn_criar_tipo.on_click(self.criar_tipo)
        self.btn_editar_tipo.on_click(self.editar_tipo)
        self.btn_remover_tipo.on_click(self.remover_tipo)
        self.btn_registrar_mov.on_click(self.registrar_movimentacao)
        self.btn_editar_mov.on_click(self.editar_movimentacao)
        self.btn_remover_mov.on_click(self.remover_movimentacao)
        self.btn_transferir.on_click(self.realizar_transferencia)
        self.btn_rel_contas.on_click(self.mostrar_contas)
        self.btn_rel_tipos.on_click(self.mostrar_tipos)
        self.btn_rel_mov.on_click(self.mostrar_movimentacoes)

        # Saídas
        self.output = widgets.Output()
        self.grafico_output = widgets.Output()

        self.atualizar_dropdowns()

    def atualizar_dropdowns(self):
        # Atualiza dropdowns de seleção
        self.conta_mov.options = [c.nome for c in self.sistema.contas]
        self.conta_origem.options = [c.nome for c in self.sistema.contas]
        self.conta_destino.options = [c.nome for c in self.sistema.contas]
        self.tipo_mov.options = [t.nome for t in self.sistema.tipos_movimentacao]

        # Atualiza dropdowns para edição/remoção
        self.dropdown_contas.options = [(f"{c.id} - {c.nome}", c.id) for c in self.sistema.contas]
        self.dropdown_tipos.options = [(f"{t.id} - {t.nome}", t.id) for t in self.sistema.tipos_movimentacao]

        # Prepara lista de movimentações para o dropdown
        mov_options = []
        for conta in self.sistema.contas:
            for mov in conta.movimentacoes:
                mov_options.append((
                    f"Conta {conta.nome}: {mov.data.strftime('%d/%m/%Y')} - {mov.tipo_movimentacao.nome} - R${mov.valor:.2f}",
                    mov.id
                ))
        self.dropdown_movimentacoes.options = mov_options

    def exibir_painel_principal(self):
        tab = widgets.Tab()
        tab.children = [
            self.criar_aba_cadastros(),
            self.criar_aba_movimentacoes(),
            self.criar_aba_transferencias(),
            self.criar_aba_relatorios()
        ]

        # Títulos
        tab.titles = [
            '📝 Cadastros',
            '💰 Movimentações',
            '🔄 Transferências',
            '📊 Relatórios'
        ]

        display(tab)
        display(self.output)
        display(self.grafico_output)

    def criar_aba_cadastros(self):
      # Contas
      secao_contas = widgets.VBox([
          widgets.HTML("<h3>Contas</h3>"),
          widgets.HBox([
              widgets.VBox([
                  widgets.HTML("<h4>Cadastrar Nova Conta</h4>"),
                  self.nome_conta,
                  self.descricao_conta,
                  self.saldo_inicial,
                  self.btn_criar_conta
              ]),
              widgets.VBox([
                  widgets.HTML("<h4>Editar/Remover Conta</h4>"),
                  self.dropdown_contas,
                  self.novo_nome_conta,
                  self.nova_descricao_conta,
                  self.novo_saldo_conta,
                  widgets.HBox([self.btn_editar_conta, self.btn_remover_conta])
              ])
          ])
      ])

      # Tipos de Movimentação
      secao_tipos = widgets.VBox([
          widgets.HTML("<h3>Tipos de Movimentação</h3>"),
          widgets.HBox([
              widgets.VBox([
                  widgets.HTML("<h4>Cadastrar Novo Tipo</h4>"),
                  self.nome_tipo,
                  self.categoria_tipo,
                  self.descricao_tipo,
                  self.btn_criar_tipo
              ]),
              widgets.VBox([
                  widgets.HTML("<h4>Editar/Remover Tipo</h4>"),
                  self.dropdown_tipos,
                  self.novo_nome_tipo,
                  self.nova_categoria_tipo,
                  self.nova_descricao_tipo,
                  widgets.HBox([self.btn_editar_tipo, self.btn_remover_tipo])
              ])
          ])
      ])

      return widgets.VBox([
          secao_contas,
          widgets.HTML("<hr>"),
          secao_tipos
      ])

    def criar_aba_movimentacoes(self):
      return widgets.VBox([
          widgets.HTML("<h3>Registrar Nova Movimentação</h3>"),
          self.conta_mov,
          self.tipo_mov,
          self.valor_mov,
          self.descricao_mov,
          self.btn_registrar_mov,

          widgets.HTML("<h3>Editar/Remover Movimentação</h3>"),
          self.dropdown_movimentacoes,
          self.novo_valor_mov,
          self.nova_descricao_mov,
          widgets.HBox([self.btn_editar_mov, self.btn_remover_mov])
      ])

    def criar_aba_transferencias(self):
        return widgets.VBox([
            widgets.HTML("<h3>Transferência entre Contas</h3>"),
            self.conta_origem,
            self.conta_destino,
            self.valor_transf,
            self.descricao_transf,
            self.btn_transferir
        ])

    def criar_aba_relatorios(self):
        return widgets.VBox([
            widgets.HTML("<h3>Relatórios</h3>"),
            widgets.HBox([self.btn_rel_contas, self.btn_rel_tipos, self.btn_rel_mov])
        ])

    # Funções
    def criar_conta(self, b):
        with self.output:
            clear_output()
            try:
                nome = self.nome_conta.value
                descricao = self.descricao_conta.value
                saldo = self.saldo_inicial.value

                conta = self.sistema.criar_conta(nome, descricao, saldo)
                if conta:
                    self.atualizar_dropdowns()
                    print(f"Conta '{nome}' criada com sucesso!")
                else:
                    print("Erro ao criar conta no banco de dados")
            except Exception as e:
                print(f"Erro ao criar conta: {str(e)}")

    def editar_conta(self, b):
        with self.output:
            clear_output()
            try:
                conta_id = self.dropdown_contas.value
                conta = self.sistema.get_conta_por_id(conta_id)

                if conta:
                    novo_nome = input("Novo nome (deixe em branco para manter): ") or conta.nome
                    nova_descricao = input("Nova descrição (deixe em branco para manter): ") or conta.descricao
                    novo_saldo = float(input("Novo saldo (deixe em branco para manter): ") or conta.saldo)

                    if self.sistema.editar_conta(conta_id, novo_nome, nova_descricao, novo_saldo):
                        print("Conta editada com sucesso!")
                        self.atualizar_dropdowns()
                    else:
                        print("Erro ao editar conta.")
                else:
                    print("Conta não encontrada!")
            except Exception as e:
                print(f"Erro ao editar conta: {str(e)}")

    def remover_conta(self, b):
        with self.output:
            clear_output()
            try:
                conta_id = self.dropdown_contas.value

                if self.sistema.remover_conta(conta_id):
                    print("Conta removida com sucesso!")
                    self.atualizar_dropdowns()
                else:
                    print("Erro ao remover conta. Verifique se não há movimentações associadas.")
            except Exception as e:
                print(f"Erro ao remover conta: {str(e)}")

    def criar_tipo(self, b):
        with self.output:
            clear_output()
            try:
                nome = self.nome_tipo.value
                categoria = CategoriaMovimentacao(self.categoria_tipo.value)
                descricao = self.descricao_tipo.value

                tipo = self.sistema.criar_tipo_movimentacao(nome, categoria, descricao)
                if tipo:
                    self.atualizar_dropdowns()
                    print(f"Tipo de movimentação '{nome}' criado com sucesso!")
                else:
                    print("Erro ao criar tipo no banco de dados")
            except Exception as e:
                print(f"Erro ao criar tipo: {str(e)}")

    def editar_tipo(self, b):
        with self.output:
            clear_output()
            try:
                tipo_id = self.dropdown_tipos.value
                tipo = self.sistema.get_tipo_movimentacao_por_id(tipo_id)

                if tipo:
                    novo_nome = input("Novo nome (deixe em branco para manter): ") or tipo.nome
                    nova_categoria = CategoriaMovimentacao(input("Nova categoria (Receita/Despesa/Transferência): ") or tipo.categoria.value)
                    nova_descricao = input("Nova descrição (deixe em branco para manter): ") or tipo.descricao

                    if self.sistema.editar_tipo_movimentacao(tipo_id, novo_nome, nova_categoria, nova_descricao):
                        print("Tipo de movimentação editado com sucesso!")
                        self.atualizar_dropdowns()
                    else:
                        print("Erro ao editar tipo de movimentação.")
                else:
                    print("Tipo não encontrado!")
            except Exception as e:
                print(f"Erro ao editar tipo: {str(e)}")

    def remover_tipo(self, b):
        with self.output:
            clear_output()
            try:
                tipo_id = self.dropdown_tipos.value

                if self.sistema.remover_tipo_movimentacao(tipo_id):
                    print("Tipo de movimentação removido com sucesso!")
                    self.atualizar_dropdowns()
                else:
                    print("Erro ao remover tipo. Verifique se não está sendo usado em movimentações.")
            except Exception as e:
                print(f"Erro ao remover tipo: {str(e)}")

    def registrar_movimentacao(self, b):
        with self.output:
            clear_output()
            try:
                conta_nome = self.conta_mov.value
                tipo_nome = self.tipo_mov.value
                valor = self.valor_mov.value
                descricao = self.descricao_mov.value

                conta = self.sistema.get_conta_por_nome(conta_nome)
                tipo = self.sistema.get_tipo_movimentacao_por_nome(tipo_nome)

                mov = MovimentacaoSimples(
                    len(conta.movimentacoes) + 1,
                    valor,
                    datetime.now(),
                    descricao,
                    conta,
                    tipo
                )

                mov.id = self.sistema.db.inserir_movimentacao(mov)
                if mov.id:
                    conta.movimentacoes.append(mov)
                    resultado = conta.creditar(valor) if tipo.categoria == CategoriaMovimentacao.RECEITA else conta.debitar(valor)
                    print(resultado)
                else:
                    print("Erro ao registrar movimentação no banco de dados")
            except Exception as e:
                print(f"Erro ao registrar movimentação: {str(e)}")

    def editar_movimentacao(self, b):
        with self.output:
            clear_output()
            try:
                mov_id = self.dropdown_movimentacoes.value

                novo_valor = float(input("Novo valor: "))
                nova_descricao = input("Nova descrição: ")

                if self.sistema.editar_movimentacao(mov_id, novo_valor, nova_descricao):
                    print("Movimentação editada com sucesso!")
                    self.atualizar_dropdowns()
                else:
                    print("Erro ao editar movimentação.")
            except Exception as e:
                print(f"Erro ao editar movimentação: {str(e)}")

    def remover_movimentacao(self, b):
        with self.output:
            clear_output()
            try:
                mov_id = self.dropdown_movimentacoes.value

                if self.sistema.remover_movimentacao(mov_id):
                    print("Movimentação removida com sucesso!")
                    self.atualizar_dropdowns()
                else:
                    print("Erro ao remover movimentação.")
            except Exception as e:
                print(f"Erro ao remover movimentação: {str(e)}")

    def realizar_transferencia(self, b):
        with self.output:
            clear_output()
            try:
                origem = self.conta_origem.value
                destino = self.conta_destino.value
                valor = self.valor_transf.value
                descricao = self.descricao_transf.value

                if not origem or not destino:
                    print("Por favor, selecione ambas as contas")
                    return
                if origem == destino:
                    print("Erro: Conta de origem e destino devem ser diferentes")
                    return
                if valor <= 0:
                    print("O valor da transferência deve ser positivo")
                    return

                resultado = self.sistema.transferir_entre_contas(origem, destino, valor, descricao)
                print(resultado)
                self.atualizar_dropdowns()
            except Exception as e:
                print(f"Erro na transferência: {str(e)}")

    def mostrar_contas(self, b):
        with self.output:
            clear_output()
            if not self.sistema.contas:
                print("Nenhuma conta cadastrada.")
                return

            dados = []
            for conta in self.sistema.contas:
                dados.append({
                    'ID': conta.id,
                    'Nome': conta.nome,
                    'Descrição': conta.descricao,
                    'Saldo': f"R${conta.saldo:.2f}",
                    'Movimentações': len(conta.movimentacoes)
                })

            display(pd.DataFrame(dados))

    def mostrar_tipos(self, b):
        with self.output:
            clear_output()
            if not self.sistema.tipos_movimentacao:
                print("Nenhum tipo de movimentação cadastrado.")
                return

            dados = []
            for tipo in self.sistema.tipos_movimentacao:
                dados.append({
                    'ID': tipo.id,
                    'Nome': tipo.nome,
                    'Categoria': tipo.categoria.value,
                    'Descrição': tipo.descricao
                })

            display(pd.DataFrame(dados))

    def mostrar_movimentacoes(self, b):
        with self.output:
            clear_output()
            if not self.sistema.contas:
                print("Nenhuma conta cadastrada.")
                return

            # Todas movimentações
            todas_movimentacoes = []
            for conta in self.sistema.contas:
                for mov in conta.movimentacoes:
                    todas_movimentacoes.append({
                        'Data': mov.data.strftime('%d/%m/%Y %H:%M'),
                        'Conta': conta.nome,
                        'Tipo': mov.tipo_movimentacao.nome,
                        'Categoria': mov.tipo_movimentacao.categoria.value,
                        'Valor': mov.valor,
                        'Descrição': mov.descricao
                    })

            if todas_movimentacoes:
                df = pd.DataFrame(todas_movimentacoes)
                display(df)

                with self.grafico_output:
                    clear_output()
                    plt.figure(figsize=(10, 5))

                    # Gráfico por categoria
                    df_grouped = df.groupby('Categoria')['Valor'].sum().reset_index()
                    plt.bar(df_grouped['Categoria'], df_grouped['Valor'])
                    plt.title("Total por Categoria")
                    plt.ylabel("Valor (R$)")
                    plt.show()
            else:
                print("Nenhuma movimentação registrada.")

In [25]:
# Iniciar
app = Financas()

Output()

Output()